# TODO
- update
    - git pull pyontutils
    - git pull ontquery
- setup ilxutils
    - cd pyontutils/ilxutils
    - pip3 install --user --editable .

# Google Sheets Import

In [14]:
from pyontutils.sheets import Sheet
import pandas as pd

class Brainstem(Sheet):
    name = 'sparc-terms' # key name you gave the google sheet id value in secrets.yaml
    sheet_name = 'Minimal information model(MIS)' # the actual sheet name on the google sheet
    fetch_grid = True # meta data in self.grid that has detials like bolding
    
brainstem = Brainstem()
df = pd.DataFrame(brainstem.raw_values)
df.head()

,0,1
0,term,curie
1,abdominal cavity,UBERON:0003684
2,abdominal wall,UBERON:0003697
3,adipose tissue,UBERON:0001013
4,adult organism,UBERON:0007023


# QUERY DATABASES 

In [1]:
from pyontutils.core import query, OntTerm

In [2]:
# Give "query" a usable parameter to query the databases 
from pyontutils.core import query # OntTerm
query(term='brain')

[OntTerm('HBA:3999', label='brain (hba)'),
 OntTerm('FMA:50801', label='Brain'),
 OntTerm('UBERON:0000955', label='brain'),
 OntTerm('UBERON:6110636', label='adult cerebral ganglion'),
 OntTerm('ILX:0101431', label='Brain')]

In [3]:
# similar entities will show
query(term='DN1 neuron')

[OntTerm('ILX:0103358', label='DN1 neuron'),
 OntTerm('ILX:0103361', label='DN2 neuron')]

In [4]:
# Faster and more accurate with curie/iri
query(curie='UBERON:0000955')

[OntTerm('UBERON:0000955', label='brain')]

In [5]:
entity = query(curie='UBERON:0000955')[0]
# Full result attribute
vars(entity)

{'prefix': 'UBERON',
 'suffix': '0000955',
 'orig_kwargs': {'iri': 'http://purl.obolibrary.org/obo/UBERON_0000955',
  'curie_or_iri': None,
  'label': None,
  'term': None,
  'search': None,
  'validated': None,
  'query': None},
 'kwargs': {'iri': 'http://purl.obolibrary.org/obo/UBERON_0000955',
  'curie_or_iri': None,
  'label': None,
  'term': None,
  'search': None,
  'validated': None,
  'query': None},
 'label': 'brain',
 'labels': ['brain'],
 'definition': 'The brain is the center of the nervous system in all vertebrate, and most invertebrate, animals. Some primitive animals such as jellyfish and starfish have a decentralized nervous system without a brain, while sponges lack any nervous system at all. In vertebrates, the brain is located in the head, protected by the skull and close to the primary sensory apparatus of vision, hearing, balance, taste, and smell[WP].',
 'synonyms': ['the brain',
  'synganglion',
  'suprasegmental structures',
  'suprasegmental levels of nervous s

# DEBUGGING HINT
- 1 "?" at the end of a function or class will return its params, docstring, and pathing. 
- 2 "??" returns the ENTIRE class/functions 

In [6]:
query?

Signature:      query(*args, **kwargs)
Call signature:
query(
    term=None,
    prefix=(),
    category=None,
    label=None,
    abbrev=None,
    search=None,
    suffix=None,
    curie=None,
    iri=None,
    predicates=(),
    exclude_prefix=(),
    depth=1,
    direction='OUTGOING',
    limit=10,
    include_deprecated=False,
)
Type:           OntQueryCli
String form:    <ontquery.query.OntQueryCli object at 0x7f78c89bef98>
File:           ~/Dropbox/git/ontquery/ontquery/query.py
Docstring:      <no docstring>


# BONUS!

### Concurrently search! (Run multiple query functions at the same time)

In [7]:
from pyontutils.utils import Async, deferred
from pyontutils.core import OntTerm, ixr, query
from typing import List, Tuple

query.setup()

def queries(kwargs_list:List[dict]) -> List[Tuple[str, dict]]:
    '''Asynchronously query databases to dramatically increase runtime un users end 
    
    Examples:
        >>> queries([{'term':'Brain'},])
        [({'term': 'Brain'},
          [OntTerm('HBA:3999', label='brain (hba)'),
           OntTerm('FMA:50801', label='Brain'),
           OntTerm('UBERON:0000955', label='brain'),
           OntTerm('UBERON:6110636', label='adult cerebral ganglion')])]
        >>> queries([{'curie':'UBERON:0000955'},])
        [({'curie': 'UBERON:0000955'}, [OntTerm('UBERON:0000955', label='brain')])]
        
    Definitions:
        kwargs == common name given to dictionary input for function
        tuple == a list that you cannot update. 
        lambda == short-hand for single line function creation (func = lambda <input>: <output>)  
        
    Args:
        kwargs_list (list): A list of dictionaries that are paramaters for the query function
    
    Returns:
        List[tuple]: A list of tuples all being of (<input>, <query_result>).    
    '''
    # create a query function wrapper to return tuple
    # kwargs -> (kwargs, query_result)
    # We do this in case 2+ queries return the same results & the output WILL NOT have the same input order
    gin = lambda kwargs: (kwargs, query(**kwargs))
    # run each query instance at the same time
    results = Async()(deferred(gin)(kwargs) for kwargs in kwargs_list)
    return results 

queries([{'curie':'UBERON:0000955'}, {'curie':'UBERON:6110636'}])

Futures compiled


[({'curie': 'UBERON:0000955'}, [OntTerm('UBERON:0000955', label='brain')]),
 ({'curie': 'UBERON:6110636'},
  [OntTerm('UBERON:6110636', label='adult cerebral ganglion')])]

# Modifing TEST InterLex

In [ ]:
# cd ~/pyontutils/ilxutils/
# pip3 install --user --editable .

In [18]:
# TEST InterLex endpoints
from ilxutils.remotes import interlex_remote_test as ixrt

# ADD

In [9]:
ixrt.add_entity?

Signature:
ixrt.add_entity(
    type,
    subThingOf,
    label,
    definition: str = None,
    synonyms=(),
    comment: str = None,
    predicates: dict = None,
)
Docstring: <no docstring>
File:      ~/Dropbox/git/ontquery/ontquery/plugins/services.py
Type:      method


In [10]:
entity = dict(
    label = 'offical label', # Can only one unique label per person
    type = 'term', # OPTIONS: term, annotation, relationship, cde, fde, pde
    definition = 'official definition',
    comment = 'helpful misc',
    # Optional
    subThingOf = '', # WARNING ::: must have at last '', can be blank but please fill this in if you can. 
    synonyms = ['Encephalon', 'Cerebro'],
    predicates = {} # annotations and/or relationships to add
    # TODO: existing_ids will be an option later
)
result = ixrt.add_entity(**entity)
print(result)

[2019-08-01 15:58:34,277] -  WARNING -       ontquery - interlex_client.py:473  - You already added entity offical label
with ILX ID: tmp_0150124


QueryResult({'iri': 'http://uri.interlex.org/base/tmp_0150124', 'curie': 'TMP:0150124', 'label': 'Offical label', 'labels': (), 'definition': 'official definition', 'synonyms': ('Encephalon', 'Cerebro'), 'deprecated': None, 'predicates': {'comment': 'helpful misc'}, 'type': None, 'types': (), '_graph': None, 'source': <ontquery.plugins.services.InterLexRemote object at 0x7f78c8677860>})


# UPDATE

In [11]:
ixrt.update_entity?

Signature:
ixrt.update_entity(
    ilx_id: str = None,
    type: str = None,
    subThingOf: str = None,
    label: str = None,
    definition: str = None,
    synonyms=(),
    comment: str = None,
    predicates_to_add: dict = None,
    predicates_to_delete: dict = None,
)
Docstring: <no docstring>
File:      ~/Dropbox/git/ontquery/ontquery/plugins/services.py
Type:      method


In [12]:
from ilxutils.remotes import interlex_remote_test as ixrt
entity = dict(
    ilx_id = 'TMP:0150124',
    label = 'Offical label', # Can only one unique label per person
    type = 'term', # OPTIONS: term, annotation, relationship, cde, fde, pde
    definition = 'official definition',
    comment = 'helpful misc',
    # Optional
    subThingOf = '', # WARNING ::: must have at last '', can be blank but please fill this in if you can. 
    synonyms = ['Encephalon', 'Cerebro'],
    predicates_to_add = {}, # annotations and/or relationships to add
    predicates_to_delete = {}, # annotations and/or relationships to del
    # TODO: existing_ids will be an option later
)
result = ixrt.update_entity(**entity)
print(result)

[2019-08-01 15:58:38,017] -     INFO -       ontquery - interlex_client.py:582  - {'ilx_id': 'TMP:0150124', 'label': 'Offical label', 'type': 'term', 'definition': 'official definition', 'comment': 'helpful misc', 'superclass': '', 'synonyms': ['Encephalon', 'Cerebro']}


QueryResult({'iri': 'http://uri.interlex.org/base/tmp_0150124', 'curie': 'TMP:0150124', 'label': 'Offical label', 'labels': (), 'definition': 'official definition', 'synonyms': ('Encephalon', 'Cerebro'), 'deprecated': None, 'predicates': {'comment': 'helpful misc'}, 'type': None, 'types': (), '_graph': None, 'source': <ontquery.plugins.services.InterLexRemote object at 0x7f78c8677860>})


# PRODUCTION 

In [17]:
# PRODUCTION 
from ilxutils.remotes import interlex_remote_production as ixr
# BE CAREFUL :)